In [ ]:
%pip install pypdf
%pip install -q transformers einops accelerate langchain
%pip install sentence_transformers
%pip install llama_index
%pip install -U langchain-community
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-langchain
%pip install chromadb
%pip install llama-index-vector-stores-chroma

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torchvision (from versions: none)
ERROR: No matching distribution found for torchvision

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: c:\Users\palan\Downloads\Diet_plan_recom_chatbot\venv\Scripts\python.exe -m pip install --upgrade pip


In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

c:\Users\palan\Downloads\Diet_plan_recom_chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\palan\Downloads\Diet_plan_recom_chatbot\venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\palan\Downloads\Diet_plan_recom_chatbot\venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\palan\Downloads\Diet_plan_recom_chatbot\venv\Lib\site-packages\pydantic\_intern

In [4]:
documents = SimpleDirectoryReader("data/disease").load_data()

In [5]:
system_prompt = """
You are tasks are to predict disease from user input and provide diet plans. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [5]:
!huggingface-cli login

^C


In [1]:
import torch
print(torch.cuda.is_available())

False


In [7]:
import torch

# meta-llama/Meta-Llama-3.1-8B
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [3]:
import torch

print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")


False
None
No GPU available


In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=1024)

In [ ]:
# """ SAVE TO LOCAL"""
db = chromadb.PersistentClient(path="./storage/chroma")
chroma_collection = db.get_or_create_collection("diseases")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
disease_index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)

In [ ]:
# """ SAVE TO LOCAL"""
chroma_collection = db.get_or_create_collection("diet")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)

In [ ]:
# # fetch documents and index from storage
# db = chromadb.PersistentClient(path="./storage/chroma")
# chroma_collection = db.get_or_create_collection("diseases1")
# chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# chroma_index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store)

In [ ]:
print(f"Available collections: {db.list_collections()}")    # <- this returns the collection that I want to get

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("What is the essential fatty acid for human?")
response.response

In [ ]:
response=query_engine.query("What is your name?")
response.response

In [ ]:
response=query_engine.query("How many times a week someone can eat fish?")
response.response

In [ ]:
response=query_engine.query("How are you trained?")
response.response